File for finding the metrics of each of the graphs at different distances.

In [2]:
import os
import re
import pandas as pd
import igraph as ig
import numpy as np
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point, LineString
import geopandas as gpd
import graph_functions as graf

In [ ]:
def download_classified_thumbnails(folder_directory, data_folder, shp_dir, 
                                   n_clusters = 10, fishnet_rows = 5, 
                                   fishnet_cols = 5, scale = 1, xmeans = False, error_log = False):
    
    # cleaning out any extra files in case ran previously
    delete_file_path = f'{shp_dir}/'
    delete_file_paths = os.listdir(delete_file_path)
    for file in delete_file_paths:
        if (re.search(r'[0-9]_gcs+', file)) or (re.search(r'[a-zA-Z]_gcs+', file)):
            full_path = f'{delete_file_path}{file}'
            print(full_path)
            os.remove(full_path)
            print(f'removed {file}')
    
    # gaining a list of all the files in the given folder
    shp_files = [f for f in os.listdir(shp_dir) if f.endswith('.shp')]

    count = 0
    # Iterate directory
    for path in os.listdir(shp_dir):
        # check if current path is a file
        if path.endswith('.shp'):
            count += 1
    print(f'Downloading data for {count} files')
    
    index_num = -1
    for shp_file in shp_files:
        # increase the index by one for file searching and for creation later
        index_num = index_num + 1
        file_name_split = shp_file.split(".")
        
        # create the path to the start shapefile (polygon area)
        shp_path = os.path.join(shp_dir, shp_file)
        # create a feature collection from the feature
        feature_collection = geemap.shp_to_ee(shp_path)

Some function examples

In [ ]:
print(f'density : {test_graph.density()}')
print(f'average path length : {test_graph.average_path_length()}')
print(f'diameter : {test_graph.diameter()}')
print(f'transitivity : {test_graph.transitivity_undirected()}')
print(f"kleinberg's hub score : {max(test_graph.hub_score())}") # will need to attach to lek utm's
print(f'cutpoints : {test_graph.articulation_points()}') # will need to attach to lek utm's

Function testing

In [ ]:
given_directory = 'E:/!!Research/!!!Data/graph_analysis/igraph_graphs/coalescence'
years_ex_list = []
type_list = []
mean_clust_info = []
num_clust_info = []
types_list = ['average', 'dispersal', 'coalescence', 'round', 'trip', '48', 'max']
adjacency_files = [f for f in os.listdir(given_directory) if f.endswith('.adjacency')]

for adj_file in adjacency_files:
    file_path = f'{given_directory}/{adj_file}'
    #parts = adj_file.split('_')
    path_parts = adj_file.split('.')
    parts = []
    for piece in path_parts:
        pieces = piece.split('_')
        parts.append(pieces)
    #print(parts[0][0])

    #Create the graph from the file
    graph = ig.Graph.Read_Adjacency(file_path)
    # Cluster stuff
    clusters = graph.connected_components()
    # number of clusters
    num_clust_info.append(len(clusters))
    # mean size of clusters
    cluster_sizes = clusters.sizes()
    print(cluster_sizes)
    mean_cluster_size = sum(cluster_sizes) / len(cluster_sizes)
    mean_clust_info.append(mean_cluster_size)
    """
    years_ex_list.append(parts[0])
    found_types = []
    for part in parts:
        if part in types_list:
            found_types.append(part)
            
    if found_types:
        type_list.append('_'.join(found_types))"""
            

#print(years_ex_list)
#print(type_list)

Global metrics

In [ ]:
def global_metrics_df(given_directory):
    # graph_type - already have a list of keywords to look for
    types_list = ['average', 'dispersal', 'disperse', 'coalescence', 'round', 'trip', '48', 'max']
    graph_type = []
    # to easily find the year
    years = []
    densitys = []
    avg_path_lengths = []
    diameters = []
    transitivitys = []
    num_clusters = []
    mean_clusters = []
    max_cluster_sizes = []

    adjacency_files = [f for f in os.listdir(f'{given_directory}') if f.endswith('.adjacency')]

    for adj_file in adjacency_files:
        file_path = f'{given_directory}/{adj_file}'
        path_parts = adj_file.split('.')
        parts = []
        for piece in path_parts:
            pieces = piece.split('_')
            parts.append(pieces)
        #parts = path_parts.split('_')
        years.append(parts[0][0])
        found_types = []
        for part in parts[0]:
            if part in types_list:
                found_types.append(part)
                
        if found_types:
            graph_type.append('_'.join(found_types))
        
        #Create the graph from the file
        graph = ig.Graph.Read_Adjacency(file_path)
        # Density
        densitys.append(graph.density())
        # Average path length
        avg_path_lengths.append(graph.average_path_length())
        # Diameter
        diameters.append(graph.diameter())
        # Undirected transitivity
        transitivitys.append(graph.transitivity_undirected())
        #
        # Cluster stuff
        clusters = graph.connected_components()
        # number of clusters
        num_clusters.append(len(clusters))
        # mean size of clusters
        cluster_sizes = clusters.sizes()
        mean_cluster_size = sum(cluster_sizes) / len(cluster_sizes)
        mean_clusters.append(mean_cluster_size)
        # Max size of a cluster
        max_cluster_sizes.append(max(cluster_sizes))
        

    data = {'graph_type' : graph_type,
                'year' : years,
                'density' : densitys,
                'avg_path_length' : avg_path_lengths,
                'diameter' : diameters,
                'transitivity' : transitivitys,
                'clusters' : num_clusters,
                'mean_cluster_size' : mean_clusters,
                'max_cluster_size' : max_cluster_sizes
                }
    df = pd.DataFrame(data)
    return df

In [ ]:
average_dispersal_df = global_metrics_df('E:/!!Research/!!!Data/graph_analysis/igraph_graphs/average_dispersal')
round_trip_df = global_metrics_df('E:/!!Research/!!!Data/graph_analysis/igraph_graphs/round_trip')
thresh48_df = global_metrics_df('E:/!!Research/!!!Data/graph_analysis/igraph_graphs/threshold_4_8')
max_disperse_df = global_metrics_df('E:/!!Research/!!!Data/graph_analysis/igraph_graphs/threshold_max_disperse')
coalescence_df = global_metrics_df('E:/!!Research/!!!Data/graph_analysis/igraph_graphs/coalescence')

frames = [average_dispersal_df, round_trip_df, thresh48_df, max_disperse_df, coalescence_df]
frame_concat = pd.concat(frames)
frame_concat.to_csv("E:/!!Research/!!!Data/graph_analysis/igraph_metrics_data/lek_rolling_binary_global_metrics.csv")

Per node metrics

Per cluster metrics